In [ ]:
# Import modules
try:
  import mediapipe as mp
except:
  !pip install mediapipe
  import mediapipe as mp
import numpy as np
import cv2 as cv
# Initialize the recognizer object and drawing pens
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
# Start using your camera
cap = cv.VideoCapture(0)
# Initialize the whiteboard
bbox_array = np.zeros([480,640,3], dtype=np.uint8)
bbox_array.fill(255)
# Initialize variables
point = (0,0)
color = (0,0,0)
size = 5
cir = False
# Initialize the edge gadgets
for i in range(0, 10):
    bbox_array[48 * i : 48 * (i + 1), : 10] = np.array([[[25.5*(i),0,0]]*10]*48)
for i in range(0, 10):
    bbox_array[48 * i : 48 * (i + 1),(-10) :] = np.array([[[0,25.5*(i),0]]*10]*48)
for i in range(0, 10):
    bbox_array[:20, 320 + 32 * i : 320 + 32 * (i + 1)] = np.array([[[0, 0, 25.5*(i)]]*32]*20)
cv.circle(bbox_array, (33,5), 4, (255, 0, 0), -1)
cv.circle(bbox_array, (33,15), 4, (255, 0, 0), -1)
bbox_array[:20, 53] = [[0, 0, 255]]*20
cv.circle(bbox_array, (115,10), 5, (0, 0, 0), -1)
cv.circle(bbox_array, (183,10), 7, (0, 0, 0), -1)
cv.circle(bbox_array, (255,10), 9, (0, 0, 0), -1)
arr = bbox_array.copy()
while True:
    # Read the data from your camera
    bbox_array = arr
    istrue, frame = cap.read()
    imgRGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            # Check if you are only using index finger to draw 
            if int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999) > 20 and 10 <= int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)) <= 630:
                mpDraw.draw_landmarks(frame, handLms)
                if ((handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_MCP].y >= handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y) == (handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_MCP].y <= handLms.landmark[mpHands.HandLandmark.WRIST].y) and not (((handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_MCP].y >= handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_TIP].y) == (handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_MCP].y <= handLms.landmark[mpHands.HandLandmark.WRIST].y)) and (handLms.landmark[mpHands.HandLandmark.RING_FINGER_MCP].y >= handLms.landmark[mpHands.HandLandmark.RING_FINGER_TIP].y) == (handLms.landmark[mpHands.HandLandmark.RING_FINGER_MCP].y <= handLms.landmark[mpHands.HandLandmark.WRIST].y))):
                    if point != (0,0):
                        # Decide if you are using dotted line or solid line
                        if not cir:
                            cv.line(bbox_array, point, (int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)),int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999)),color,size)
                        else:
                            cv.circle(bbox_array, ((int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)),int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999))), size, color, -1)
                    # Initialize a point to start the line
                    point = (int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)),int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999))
                # Check if you are using 4 fingers to erase the whiteboard
                if ((handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_MCP].y >= handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_TIP].y) == (handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_MCP].y <= handLms.landmark[mpHands.HandLandmark.WRIST].y) and (handLms.landmark[mpHands.HandLandmark.RING_FINGER_MCP].y >= handLms.landmark[mpHands.HandLandmark.RING_FINGER_TIP].y) == (handLms.landmark[mpHands.HandLandmark.RING_FINGER_MCP].y <= handLms.landmark[mpHands.HandLandmark.WRIST].y)):
                    cv.circle(bbox_array, ((int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)),int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999))), (10), (255,255,255), -1)
                    cv.circle(bbox_array, ((int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)),int(handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_TIP].y*479.9999))), (10), (255,255,255), -1)
            else:
                if int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999) < 20 and int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)) < 320: 
                    # Change the line to dotted
                    if 10 < int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)) <= 40:
                        cir = True
                        point = (0,0)
                    # Change the line to solid
                    elif 40 < int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)) <= 90:
                        cir = False
                        point = (0,0)
                    # Change the size of the line
                    elif 90 < int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)) <= 147:
                        size = 5
                    elif 147 < int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)) <= 218:
                        size = 7
                    elif 218 < int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)) <= 320:
                        size = 9
                else:
                    try:
                        # Change the color of the line
                        a = tuple(bbox_array[int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999), int(639*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x))])
                        color = (int(a[0]), int(a[1]), int(a[2]))
                    except:
                        pass
    # A grey tracker for the user to see their current index finger location
    arr = bbox_array.copy()
    imgRGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            cv.circle(bbox_array, ((int(639.9999*(1-handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x)),int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y*479.9999))), (5), (200,200,200), -1)
    # Show the whiteboard
    cv.imshow('whiteboard', bbox_array)
    # Show the camera with your hands
    cv.imshow('camera', frame)
    # End your code by pressing 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cv.destroyAllWindows()

